In [1]:
import tqdm
import numpy as np
import pandas as pd
import os

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from torchvision.transforms import ToTensor
import torchvision.transforms as tt

from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

%matplotlib inline

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080 Ti'

In [3]:
device = torch.device(0) if torch.cuda.is_available() else torch.device("cpu")

In [4]:
device

device(type='cuda', index=0)

# 이미지 데이터셋

In [5]:
test_data = pd.read_csv("/home/enfycius/Documents/image_sample.csv", header=None)

In [6]:
test_data

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,71,75,266,73,621,73,833,73,1045,72,...,442,4829,445,5026,446,5346,447,5517,449,Assembled
1,74,78,266,78,617,81,827,82,1038,82,...,443,4823,442,5018,445,5342,447,5519,449,Assembled
2,73,76,258,79,597,77,798,74,1002,77,...,437,4799,440,5002,442,5338,443,5521,442,Assembled
3,73,72,261,73,603,73,811,73,1015,75,...,447,4807,447,5006,450,5336,451,5518,451,Assembled
4,73,61,263,64,611,66,822,66,1027,64,...,442,4815,442,5017,444,5339,445,5517,448,Assembled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,62,82,247,84,591,77,796,79,1003,77,...,449,4801,452,4998,451,5327,454,5508,452,Not Assembled
82,61,68,241,67,577,68,778,68,981,69,...,454,4779,457,4983,457,5320,461,5505,463,Not Assembled
83,63,71,244,73,584,69,786,67,989,71,...,451,4785,454,4989,453,5322,458,5507,458,Not Assembled
84,80,95,265,92,608,94,815,91,1022,94,...,469,4812,470,5014,467,5341,467,5524,469,Not Assembled


# 생성한 데이터셋

In [7]:
normal = pd.read_csv("/root/normal.csv", header=None)
abnormal = pd.read_csv("/root/abnormal.csv", header=None)

In [8]:
normal = normal.drop(0)
abnormal = abnormal.drop(0)

In [9]:
normal

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
1,74,32,250,32,581,32,782,32,978,32,...,430,4773,430,4973,430,5300,430,5481,430,Assembled
2,73,32,250,32,583,32,780,32,979,32,...,430,4773,430,4970,430,5302,430,5482,430,Assembled
3,73,32,250,32,583,32,781,32,980,32,...,430,4771,430,4970,430,5303,430,5479,430,Assembled
4,71,32,252,32,583,32,780,32,979,32,...,430,4774,430,4970,430,5300,430,5481,430,Assembled
5,71,32,252,32,583,32,780,32,979,32,...,430,4772,430,4973,430,5301,430,5481,430,Assembled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,73,32,252,32,582,32,781,32,980,32,...,430,4773,430,4973,430,5303,430,5481,430,Assembled
49997,71,32,251,32,582,32,782,32,978,32,...,430,4771,430,4971,430,5303,430,5479,430,Assembled
49998,72,32,253,32,582,32,782,32,981,32,...,430,4772,430,4972,430,5300,430,5479,430,Assembled
49999,73,32,251,32,583,32,781,32,980,32,...,430,4772,430,4970,430,5302,430,5481,430,Assembled


In [10]:
abnormal

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
1,74,32,248,32,586,32,779,32,977,32,...,430,4777,430,4980,430,5306,430,5477,430,Not Assembled
2,75,32,252,32,588,32,781,32,982,32,...,430,4774,430,4976,430,5298,430,5484,430,Not Assembled
3,78,32,250,32,575,32,792,32,976,32,...,430,4781,430,4986,430,5304,430,5474,430,Not Assembled
4,70,32,258,32,579,32,783,32,987,32,...,430,4775,430,4981,430,5299,430,5489,430,Not Assembled
5,73,32,256,32,579,32,784,32,978,32,...,430,4778,430,4968,430,5299,430,5485,430,Not Assembled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,65,32,244,32,582,32,775,32,978,32,...,430,4786,430,4982,430,5306,430,5473,430,Not Assembled
49997,70,32,246,32,595,32,780,32,991,32,...,430,4781,430,4968,430,5294,430,5479,430,Not Assembled
49998,85,32,254,32,580,32,778,32,990,32,...,430,4787,430,4967,430,5300,430,5480,430,Not Assembled
49999,90,32,270,32,598,32,774,32,985,32,...,430,4769,430,4962,430,5300,430,5479,430,Not Assembled


In [11]:
data = pd.concat([normal, abnormal], ignore_index = True)

In [12]:
data

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,74,32,250,32,581,32,782,32,978,32,...,430,4773,430,4973,430,5300,430,5481,430,Assembled
1,73,32,250,32,583,32,780,32,979,32,...,430,4773,430,4970,430,5302,430,5482,430,Assembled
2,73,32,250,32,583,32,781,32,980,32,...,430,4771,430,4970,430,5303,430,5479,430,Assembled
3,71,32,252,32,583,32,780,32,979,32,...,430,4774,430,4970,430,5300,430,5481,430,Assembled
4,71,32,252,32,583,32,780,32,979,32,...,430,4772,430,4973,430,5301,430,5481,430,Assembled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,65,32,244,32,582,32,775,32,978,32,...,430,4786,430,4982,430,5306,430,5473,430,Not Assembled
99996,70,32,246,32,595,32,780,32,991,32,...,430,4781,430,4968,430,5294,430,5479,430,Not Assembled
99997,85,32,254,32,580,32,778,32,990,32,...,430,4787,430,4967,430,5300,430,5480,430,Not Assembled
99998,90,32,270,32,598,32,774,32,985,32,...,430,4769,430,4962,430,5300,430,5479,430,Not Assembled


# 공통

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [14]:
# min_max_scaler = MinMaxScaler()

In [15]:
# std_scaler = StandardScaler()

In [16]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]

In [17]:
X

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,74,32,250,32,581,32,782,32,978,32,...,4574,430,4773,430,4973,430,5300,430,5481,430
1,73,32,250,32,583,32,780,32,979,32,...,4572,430,4773,430,4970,430,5302,430,5482,430
2,73,32,250,32,583,32,781,32,980,32,...,4573,430,4771,430,4970,430,5303,430,5479,430
3,71,32,252,32,583,32,780,32,979,32,...,4573,430,4774,430,4970,430,5300,430,5481,430
4,71,32,252,32,583,32,780,32,979,32,...,4574,430,4772,430,4973,430,5301,430,5481,430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,65,32,244,32,582,32,775,32,978,32,...,4580,430,4786,430,4982,430,5306,430,5473,430
99996,70,32,246,32,595,32,780,32,991,32,...,4581,430,4781,430,4968,430,5294,430,5479,430
99997,85,32,254,32,580,32,778,32,990,32,...,4579,430,4787,430,4967,430,5300,430,5480,430
99998,90,32,270,32,598,32,774,32,985,32,...,4576,430,4769,430,4962,430,5300,430,5479,430


In [18]:
X_b = X

In [19]:
# X = min_max_scaler.fit_transform(X.values.T)

In [20]:
# X = std_scaler.fit_transform(X.values.T)

In [21]:
# X = X.T

In [22]:
# X

In [23]:
# X = X.reshape(-1, 150)

In [24]:
# X = pd.DataFrame(X, columns=X_b.columns, index=list(X_b.index.values))

In [25]:
# X

In [26]:
TRAIN_PATH = "/home/enfycius/Desktop/train_v2/"

In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [29]:
y = encoder.transform(y)

In [30]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [31]:
print(encoder.classes_)

['Assembled' 'Not Assembled']


In [32]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [33]:
X = torch.tensor(X.values, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1).to(device)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

In [36]:
import torch
import torch.nn as nn

In [37]:
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        # self.batch1 = nn.BatchNorm1d(num_features=150)
        # self.layer1 = nn.Linear(150, 512)
        # self.act1 = nn.ReLU()
        # self.batch2 = nn.BatchNorm1d(num_features=512)
        # self.layer2 = nn.Linear(512, 2048)
        # self.act2 = nn.ReLU()
        # self.batch3 = nn.BatchNorm1d(num_features=2048)
        # self.layer3 = nn.Linear(2048, 16384)
        # self.act3 = nn.ReLU()
        # self.batch4 = nn.BatchNorm1d(num_features=16384)
        # self.layer4 = nn.Linear(16384, 8192)
        # self.act4 = nn.ReLU()
        # self.batch5 = nn.BatchNorm1d(num_features=8192)
        # self.layer5 = nn.Linear(8192, 4096)
        # self.act5 = nn.ReLU()
        # self.batch6 = nn.BatchNorm1d(num_features=4096)

        self.flatten = nn.Flatten()
        self.batch1 = nn.BatchNorm1d(num_features=150)
        self.layer1 = nn.Linear(150, 512)
        self.act1 = nn.ReLU()
        self.batch2 = nn.BatchNorm1d(num_features=512)
        self.layer2 = nn.Linear(512, 2048)
        self.act2 = nn.ReLU()
        self.batch3 = nn.BatchNorm1d(num_features=2048)
        self.layer3 = nn.Linear(2048, 1192)
        self.act3 = nn.ReLU()
        self.batch4 = nn.BatchNorm1d(num_features=1192)
        self.layer4 = nn.Linear(1192, 512)
        self.act4 = nn.ReLU()
        self.batch5 = nn.BatchNorm1d(num_features=512)
        self.layer5 = nn.Linear(512, 128)
        self.act5 = nn.ReLU()
        self.batch6 = nn.BatchNorm1d(num_features=128)
        # self.layer6 = nn.Linear(4096, 2048)
        # self.act6 = nn.ReLU()
        # self.batch7 = nn.BatchNorm1d(num_features=2048)
        # self.layer7 = nn.Linear(2048, 1024)
        # self.act7 = nn.ReLU()
        # self.batch8 = nn.BatchNorm1d(num_features=1024)
        # self.layer8 = nn.Linear(1024, 512)
        # self.act8 = nn.ReLU()
        # self.batch9 = nn.BatchNorm1d(num_features=512)
        # self.layer9 = nn.Linear(512, 256)
        # self.act9 = nn.ReLU()
        # self.batch10 = nn.BatchNorm1d(num_features=256)
        # self.layer10 = nn.Linear(256, 128)
        # self.act10 = nn.ReLU()
        # self.batch11 = nn.BatchNorm1d(num_features=128)
        self.output = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.batch1(x)
        x = self.act1(self.layer1(x))
        x = self.batch2(x)
        x = self.act2(self.layer2(x))
        x = self.batch3(x)
        x = self.act3(self.layer3(x))
        x = self.batch4(x)
        # x = self.batchnorm1(x)
        x = self.act4(self.layer4(x))
        x = self.batch5(x)
        x = self.act5(self.layer5(x))
        x = self.batch6(x)
        # x = self.batch6(x)
        # x = self.act6(self.layer6(x))
        # x = self.batch7(x)
        # x = self.act7(self.layer7(x))
        # x = self.batch8(x)
        # x = self.act8(self.layer8(x))
        # x = self.batch9(x)
        # x = self.act9(self.layer9(x))
        # x = self.batch10(x)
        # x = self.act10(self.layer10(x))
        # x = self.batch11(x)
        x = self.sigmoid(self.output(x))

        return x

In [38]:
model = Multiclass().to(device)

In [39]:
import torch.optim as optim

In [40]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-05)
# optimizer = optim.SGD(model.parameters(), lr=1e-12, momentum=0.9)

In [41]:
n_epochs = 100
batch_size = 100
batch_start = torch.arange(0, len(X_train), batch_size)

In [42]:
best_acc = -np.inf
best_weights = None

train_loss_hist = []
train_acc_hist = []
test_loss_hist = []
test_acc_hist = []

In [43]:
import copy

In [44]:
for epoch in range(n_epochs):
    epoch_loss = []
    epoch_acc = []

    model.train()

    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        
        for i in bar:
            start = i * batch_size
            X_batch = X_train[start:start + batch_size]
            y_batch = y_train[start:start + batch_size]
            y_pred = model(X_batch).to(device)

            loss = loss_fn(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = (y_pred.round() == y_batch).float().mean()
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))

            bar.set_postfix(
                loss = float(loss),
                acc = float(acc)
            )
    
    model.eval()

    y_pred = model(X_test)
    ce = loss_fn(y_pred, y_test)

    acc = (y_pred.round() == y_test).float().mean()
    ce = float(ce)
    acc = float(acc)

    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    test_loss_hist.append(ce)
    test_acc_hist.append(acc)

    if acc > best_acc:
        best_acc = acc
        best_weights = copy.deepcopy(model.state_dict())

    print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy = {acc * 100:.1f}%")

Epoch 0 validation: Cross-entropy=2.24, Accuracy = 50.0%
Epoch 1 validation: Cross-entropy=5.07, Accuracy = 50.0%
Epoch 2 validation: Cross-entropy=7.22, Accuracy = 50.0%
Epoch 3 validation: Cross-entropy=5.86, Accuracy = 50.0%
Epoch 4 validation: Cross-entropy=6.83, Accuracy = 50.0%
Epoch 5 validation: Cross-entropy=8.75, Accuracy = 50.0%
Epoch 6 validation: Cross-entropy=8.90, Accuracy = 50.0%
Epoch 7 validation: Cross-entropy=7.39, Accuracy = 50.0%
Epoch 8 validation: Cross-entropy=5.34, Accuracy = 50.0%
Epoch 9 validation: Cross-entropy=3.41, Accuracy = 50.0%
Epoch 10 validation: Cross-entropy=1.94, Accuracy = 51.0%
Epoch 11 validation: Cross-entropy=1.04, Accuracy = 55.7%
Epoch 12 validation: Cross-entropy=0.65, Accuracy = 65.4%
Epoch 13 validation: Cross-entropy=0.56, Accuracy = 76.8%
Epoch 14 validation: Cross-entropy=0.58, Accuracy = 74.6%
Epoch 15 validation: Cross-entropy=0.57, Accuracy = 74.0%
Epoch 16 validation: Cross-entropy=0.54, Accuracy = 79.5%
Epoch 17 validation: Cro

In [45]:
model = Multiclass().to(device)

In [46]:
model.load_state_dict(best_weights)

<All keys matched successfully>

In [47]:
model.eval()

Multiclass(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (batch1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Linear(in_features=150, out_features=512, bias=True)
  (act1): ReLU()
  (batch2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer2): Linear(in_features=512, out_features=2048, bias=True)
  (act2): ReLU()
  (batch3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer3): Linear(in_features=2048, out_features=1192, bias=True)
  (act3): ReLU()
  (batch4): BatchNorm1d(1192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer4): Linear(in_features=1192, out_features=512, bias=True)
  (act4): ReLU()
  (batch5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer5): Linear(in_features=512, out_features=128, bias=True)
  (act5): ReLU()
  (batch6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=

In [48]:
test_X = test_data.iloc[:, 0:-1]
test_y = test_data.iloc[:, -1]

In [49]:
encoder = LabelEncoder()
encoder.fit(test_y)

LabelEncoder()

In [50]:
test_y = encoder.transform(test_y)

In [51]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [52]:
print(encoder.classes_)

['Assembled' 'Not Assembled']


In [53]:
test_X = torch.tensor(test_X.values, dtype=torch.float32).to(device)
test_y = torch.tensor(test_y, dtype=torch.float32).reshape(-1, 1).to(device)

In [54]:
test_loss_hist = []
test_acc_hist = []

In [57]:
y_pred = model(test_X)
ce = loss_fn(y_pred, test_y)

acc = (y_pred.round() == test_y).float().mean()
ce = float(ce)
acc = float(acc)

test_loss_hist.append(ce)
test_acc_hist.append(acc)

print(f"Test: Cross-entropy={ce:.2f}, Accuracy = {acc * 100:.1f}%")

Test: Cross-entropy=58.14, Accuracy = 41.9%
